<a href="https://colab.research.google.com/github/elemnurguner/data-ai-projects/blob/main/%C3%96neri_Sistemleri_(Recommendation_Systems).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

c. Öneri Sistemleri (Recommendation Systems)
•	Amaç: Kullanıcılara ürün veya film önerisi yapma.
•	Veri Seti: MovieLens veya Amazon Product Reviews.
•	Adımlar:
1.	Veriyi yükle ve temizle.
2.	Collaborative Filtering veya Matrix Factorization teknikleri kullan.
3.	Modeli değerlendir ve öneriler yap.


In [ ]:
import numpy as np
np.__version__  # 1.23.5 olup olmadığını kontrol edin

import pandas as pd
from surprise import Dataset, Reader, KNNBasic, SVD
from surprise.model_selection import cross_validate, train_test_split
from collections import defaultdict

print("Kütüphaneler başarıyla yüklendi!")

Kütüphaneler başarıyla yüklendi!



2. Veri Setini Yükleme ve İnceleme


In [ ]:
# MovieLens küçük veri setini yükleyelim
!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

# Verileri yükleme
movies = pd.read_csv('ml-latest-small/movies.csv')
ratings = pd.read_csv('ml-latest-small/ratings.csv')
tags = pd.read_csv('ml-latest-small/tags.csv')

# İlk birkaç satırı göster
print("Filmler:")
print(movies.head())
print("\nDeğerlendirmeler:")
print(ratings.head())

--2025-03-24 21:21:43--  http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  3.28MB/s    in 0.3s    

2025-03-24 21:21:44 (3.28 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  
Filmler:
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1

3. Veri Ön İşleme


In [ ]:
# Gereksiz sütunları kaldırma
ratings = ratings.drop('timestamp', axis=1)

# Film başlıklarını ve türlerini birleştirme
movie_ratings = pd.merge(ratings, movies, on='movieId')

# Kullanıcı ve film sayılarını kontrol etme
n_users = ratings.userId.nunique()
n_movies = ratings.movieId.nunique()
print(f"Toplam {n_users} kullanıcı ve {n_movies} film bulunmaktadır.")

Toplam 610 kullanıcı ve 9724 film bulunmaktadır.


4. Collaborative Filtering Modeli (KNN)


In [ ]:
# Surprise kütüphanesi için veri formatını ayarlama
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

# KNN modelini oluşturma ve değerlendirme
knn_model = KNNBasic(sim_options={'name': 'cosine', 'user_based': False})
cross_validate(knn_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9723  0.9776  0.9768  0.9766  0.9785  0.9764  0.0022  
MAE (testset)     0.7574  0.7616  0.7610  0.7580  0.7644  0.7605  0.0025  
Fit time          12.45   8.03    7.81    7.51    7.46    8.65    1.91    
Test time         7.90    7.38    8.28    7.93    8.21    7.94    0.32    


{'test_rmse': array([0.97225819, 0.97764003, 0.9768277 , 0.97661243, 0.97851202]),
 'test_mae': array([0.75744518, 0.76159312, 0.7610116 , 0.75798934, 0.76440977]),
 'fit_time': (12.44643259048462,
  8.027082681655884,
  7.80964469909668,
  7.507258176803589,
  7.462937831878662),
 'test_time': (7.898759841918945,
  7.375197172164917,
  8.28241753578186,
  7.927589416503906,
  8.210338592529297)}

5. Matrix Factorization Modeli (SVD)


In [ ]:
# SVD modelini oluşturma ve değerlendirme
svd_model = SVD()
cross_validate(svd_model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Modeli eğitme
trainset = data.build_full_trainset()
svd_model.fit(trainset)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8622  0.8844  0.8788  0.8683  0.8667  0.8721  0.0082  
MAE (testset)     0.6642  0.6805  0.6755  0.6672  0.6659  0.6707  0.0063  
Fit time          2.64    1.69    1.60    1.69    1.58    1.84    0.40    
Test time         0.41    0.13    0.29    0.13    0.12    0.22    0.11    


6. Tavsiyeler Oluşturma


In [ ]:
def get_top_n_recommendations(user_id, n=10):
    # Kullanıcının henüz derecelendirmediği filmleri bul
    user_movies = ratings[ratings['userId'] == user_id]['movieId']
    all_movies = ratings['movieId'].unique()
    unseen_movies = [movie for movie in all_movies if movie not in user_movies]

    # Tahminler yap
    predictions = [svd_model.predict(user_id, movie) for movie in unseen_movies]

    # En yüksek tahmin edilen derecelendirmelere göre sırala
    predictions.sort(key=lambda x: x.est, reverse=True)

    # En iyi n filmi al
    top_n = predictions[:n]

    # Film bilgilerini getir
    top_n_movies = []
    for pred in top_n:
        movie_info = movies[movies['movieId'] == pred.iid].iloc[0]
        top_n_movies.append({
            'movieId': pred.iid,
            'title': movie_info['title'],
            'genres': movie_info['genres'],
            'estimated_rating': pred.est
        })

    return pd.DataFrame(top_n_movies)

# Örnek bir kullanıcı için tavsiyeler oluştur
user_id = 1
recommendations = get_top_n_recommendations(user_id)
print(f"Kullanıcı {user_id} için film önerileri:")
print(recommendations)

Kullanıcı 1 için film önerileri:
   movieId                                              title  \
0      457                               Fugitive, The (1993)   
1      527                            Schindler's List (1993)   
2     1136             Monty Python and the Holy Grail (1975)   
3     1196  Star Wars: Episode V - The Empire Strikes Back...   
4     1198  Raiders of the Lost Ark (Indiana Jones and the...   
5     1278                          Young Frankenstein (1974)   
6     2329                          American History X (1998)   
7     2959                                  Fight Club (1999)   
8    48516                               Departed, The (2006)   
9    58559                            Dark Knight, The (2008)   

                        genres  estimated_rating  
0                     Thriller               5.0  
1                    Drama|War               5.0  
2     Adventure|Comedy|Fantasy               5.0  
3      Action|Adventure|Sci-Fi               5.

7. Modeli Kaydetme ve Yükleme (Opsiyonel)

In [ ]:
import pickle

# Modeli kaydet
with open('movie_recommender.pkl', 'wb') as f:
    pickle.dump(svd_model, f)

# Modeli yükle
with open('movie_recommender.pkl', 'rb') as f:
    loaded_model = pickle.load(f)